In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:

pd.read_sql('SELECT 1', con=engine)


,?column?
0,1


In [3]:
query_1 = '''
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
'''

books_after_2000 = pd.read_sql(query_1, con=engine)
display(books_after_2000)


,books_after_2000
0,819


## Se identificaron 819 libros publicados después del 1 de enero de 2000

In [4]:
query_2 = '''
SELECT
    b.book_id,
    b.title,
    COUNT(DISTINCT r.review_id) AS num_reviews,
    AVG(rt.rating) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY b.book_id
'''

book_reviews_ratings = pd.read_sql(query_2, con=engine)
display(book_reviews_ratings.head(10))



,book_id,title,num_reviews,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
5,6,1st to Die (Women's Murder Club #1),4,3.500000
6,7,2nd Chance (Women's Murder Club #2),3,3.000000
7,8,4th of July (Women's Murder Club #4),3,4.000000
8,9,A Beautiful Mind,2,4.250000
9,10,A Bend in the Road,3,3.200000


In [5]:
query_3 = '''
SELECT
    publishers.publisher,
    COUNT(books.book_id) AS num_books_over_50_pages
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books_over_50_pages DESC
LIMIT 1;
'''

top_publisher = pd.read_sql(query_3, con=engine)
display(top_publisher)


,publisher,num_books_over_50_pages
0,Penguin Books,42


In [6]:
query_4 = '''
SELECT
    authors.author,
    AVG(ratings.rating) AS avg_rating
FROM books
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
GROUP BY authors.author
HAVING COUNT(ratings.rating) >= 50
ORDER BY avg_rating DESC
LIMIT 1;
'''

top_author = pd.read_sql(query_4, con=engine)
display(top_author)


,author,avg_rating
0,Diana Gabaldon,4.3


In [ ]:
query_5 = '''
with users_with_50_plus_ratings AS (
    SELECT
        username,
        COUNT(DISTINCT book_id) AS num_rated_books
    FROM ratings
    GROUP BY username
    HAVING COUNT(DISTINCT book_id) > 50
),
user_review_counts AS (
    SELECT
        username,
        COUNT(review_id) AS num_reviews
    FROM reviews
    GROUP BY username
)

SELECT
    AVG(num_reviews) AS avg_reviews_per_user
FROM user_review_counts
WHERE username IN (SELECT username FROM users_with_50_plus_ratings);
'''

avg_reviews = pd.read_sql(query_5, con=engine)
display(avg_reviews)


## Tarea 1: 
-Se encontraron 819 libros publicados después del 1 de enero de 2000, indicando un volumen importante de publicaciones recientes.
## Tarea 2: 
-Se identificó el número de reseñas y la calificación promedio para cada libro, lo que permite entender la recepción de los usuarios.
## Tarea 3: 
-La editorial que ha publicado el mayor número de libros con más de 50 páginas es Penguin Books, destacando su importancia en el catálogo.
## Tarea 4: 
-La autora con la calificación promedio más alta, considerando solo libros con al menos 50 calificaciones, es Diana Gabaldon con una calificación promedio de 4.3, indicando una alta valoración.
## Tarea 5: 
-Los usuarios que calificaron más de 50 libros escriben en promedio 24.33 reseñas, reflejando un nivel alto de participación.